# Exercise 04 : Register Model / Deploy Model

In this exercise, we will create a trained model and

- Register this model in Azure ML model management
- Deploy this model as web service in Azure

*back to [index](https://github.com/tsmatz/mlflow-azureml/)*

## 1. Create a trained model

First, set AML workspace for MLflow tracking URI.

In [1]:
from azureml.core import Workspace
import mlflow

ws = Workspace.get(
    name = "<FILL-AML-WORKSPACE-NAME>",
    subscription_id = "<FILL-AZURE-SUBSCRIPTION-ID>",
    resource_group = "<FILL-RESOUCE-GROUP-NAME>")
tracking_uri = ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(tracking_uri)

Set experiment name.

In [2]:
experimentName = "exercise04-generate-model"
mlflow.set_experiment(experimentName)

2022/03/10 06:58:56 INFO mlflow.tracking.fluent: Experiment with name 'exercise04-generate-model' does not exist. Creating a new experiment.


<Experiment: artifact_location='', experiment_id='ab85e6ca-1b21-436b-821d-055b04fc6c8c', lifecycle_stage='active', name='exercise04-generate-model', tags={}>

Run training and generate model.

In [3]:
import os
import sys

import pandas as pd
from sklearn.linear_model import ElasticNet
import mlflow

alpha=0.2
l1_ratio=0.9

# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
data = pd.read_csv(csv_url, sep=";")

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = data.drop(["quality"], axis=1)
train_y = data[["quality"]]

mlflow.sklearn.autolog()

with mlflow.start_run() as my_run:
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
mlflow.end_run()

print("completed training with alpha={} l1_ratio={}".format(alpha, l1_ratio))

completed training with alpha=0.2 l1_ratio=0.9


After the training has completed, please make sure that the trained model is in ```model"``` folder.

![Model output](./images/model_output.png)

## 2. Register model

Register model in Azure ML model management.<br>
After registration, you can manually deploy model using Python SDK, CLI/YAML, or Azure ML studio UI. (Here I just register a model.)

In [4]:
model_path = "model"
model_uri = 'runs:/{}/{}'.format(my_run.info.run_id, model_path)
mlflow.register_model(model_uri, "registered_model_test")

Successfully registered model 'registered_model_test'.
2022/03/10 06:59:20 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: registered_model_test, version 1
Created version '1' of model 'registered_model_test'.


<ModelVersion: creation_timestamp=1646895559953, current_stage='None', description='', last_updated_timestamp=1646895559953, name='registered_model_test', run_id='17f8fa2b-207d-4d1a-9d77-89512bf2511f', run_link='', source='azureml://experiments/exercise04-generate-model/runs/17f8fa2b-207d-4d1a-9d77-89512bf2511f/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

Please make sure that model is registered in Azure ML model management. (Select "Models" in [Azure ML studio UI](https://ml.azure.com/).)

![Registered model](./images/registered_model.png)

## 3. Deploy model

You can deploy the trained model as web service in Azure.<br>
In this exercise, I will deploy the above model in Azure ML managed realtime endpoint (v2 endpoint).

In MLflow deployment, the entry script (scoring script) is automatically generated by using the model specification.<br>
In order for making the model deployment enable, you should then log your model with a signature, which is the specification document, such as, inputs spec, outputs spec, etc.<br>
In scikit-learn model, the signature can be automatically inferred and saved as ```MLmodel``` file, but here I explicitly define the signature as follows.<br>
In this example, this model (with signature) is saved in ```model_with_sig``` folder.

In [5]:
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

input_schema = Schema([
  ColSpec("double", "fixed acidity"),
  ColSpec("double", "volatile acidity"),
  ColSpec("double", "citric acid"),
  ColSpec("double", "residual sugar"),
  ColSpec("double", "chlorides"),
  ColSpec("double", "free sulfur dioxide"),
  ColSpec("double", "total sulfur dioxide"),
  ColSpec("double", "density"),
  ColSpec("double", "pH"),
  ColSpec("double", "sulphates"),
  ColSpec("double", "alcohol"),
])
output_schema = Schema([ColSpec("integer")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

mlflow.sklearn.autolog()

with mlflow.start_run() as my_run:
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    model_path_with_sig = "model_with_sig"
    mlflow.sklearn.log_model(lr, model_path_with_sig, signature=signature)

mlflow.end_run()

Here we define deployment configuration in ```v2_deploy_config.json``` file.

In this configuration, I define configuration for Azure ML managed realtime endpoint (v2 endpoint).<br>
See [here](https://github.com/tsmatz/azureml-tutorial/blob/master/cli_yaml/exercise08_publish_model.ipynb) for v2 deployment configuration parameters. (The traffic will be automatically allocated as 100 %.)

In [6]:
%%writefile v2_deploy_config.json
{
    "type": "managed",
    "name": "mlflow-deploy-test",
    "instance_type": "Standard_DS2_v2",
    "instance_count": 1
}

Writing v2_deploy_config.json


Now let's deploy your model in Azure ML managed endpoint with MLflow API.<br>
Before deployment, the model is also registered in Azure ML model management.

In [7]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client(mlflow.get_tracking_uri())
test_config = {"deploy-config-file": "v2_deploy_config.json"}
client.create_deployment(
    model_uri='runs:/{}/{}'.format(my_run.info.run_id, model_path_with_sig),
    config=test_config,
    name="managed-endpoint-test")

Successfully registered model 'managed-endpoint-test-model'.
2022/03/10 07:00:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: managed-endpoint-test-model, version 1
Created version '1' of model 'managed-endpoint-test-model'.


{'id': '/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AzureML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/onlineEndpoints/managed-endpoint-test',
 'name': 'managed-endpoint-test',
 'type': 'Microsoft.MachineLearningServices/workspaces/onlineEndpoints',
 'properties': {'description': None,
  'properties': {'azureml.mlflow_client_endpoint': 'True',
   'azureml.onlineendpointid': '/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/azureml-rg/providers/microsoft.machinelearningservices/workspaces/ws01/onlineendpoints/managed-endpoint-test',
   'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:40c749df-66a2-4080-86e6-25acecf8e129:7e7cf27a-0c25-4d78-8642-656f09aa3e42?api-version=2021-10-01'},
  'scoringUri': 'https://managed-endpoint-test.eastus.inference.ml.azure.com/score',
  'swaggerUri': 'https

After the deployment has succeeded, you can invoke a web service with MLflow client API.

In [8]:
import numpy as np

wine_df = pd.DataFrame(
  np.array(
    [
      [7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4],
      [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]
    ]),
  columns=[
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol"
  ])
result_df = client.predict("managed-endpoint-test", wine_df)
result_df

,0
0,5.486353
1,5.481393


You can directly invoke by HTTP with serving URI.

In [9]:
deployinfo = client.get_deployment("managed-endpoint-test")
print("Scoring URI : {}".format(deployinfo["properties"]["scoringUri"]))
print("Swagger URI : {}".format(deployinfo["properties"]["swaggerUri"]))

Scoring URI : https://managed-endpoint-test.eastus.inference.ml.azure.com/score
Swagger URI : https://managed-endpoint-test.eastus.inference.ml.azure.com/swagger.json
